In [6]:
import os
import keras

In [8]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import itertools

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
import matplotlib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

train = pd.read_csv('./data/train.csv')
print('Shape of the train data with all features:', train.shape)
train = train.select_dtypes(exclude=['object'])
print("")
print('Shape of the train data with numerical features:', train.shape)
train.drop('Id',axis = 1, inplace = True)
train.fillna(0,inplace=True)

test = pd.read_csv('./data/test.csv')
test = test.select_dtypes(exclude=['object'])
ID = test.Id
test.fillna(0,inplace=True)
test.drop('Id',axis = 1, inplace = True)

print("")
print("List of features contained our dataset:",list(train.columns))

Shape of the train data with all features: (1460, 81)

Shape of the train data with numerical features: (1460, 38)

List of features contained our dataset: ['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice']


In [246]:
'''from sklearn.ensemble import IsolationForest

clf = IsolationForest(max_samples = 100, random_state = 42)
clf.fit(train)
y_noano = clf.predict(train)
y_noano = pd.DataFrame(y_noano, columns = ['Top'])
y_noano[y_noano['Top'] == 1].index.values

train = train.iloc[y_noano[y_noano['Top'] == 1].index.values]
train.reset_index(drop = True, inplace = True)
print("Number of Outliers:", y_noano[y_noano['Top'] == -1].shape[0])
print("Number of rows without outliers:", train.shape[0])
'''

Number of Outliers: 146
Number of rows without outliers: 1314


In [9]:
import warnings
warnings.filterwarnings('ignore')

col_train = list(train.columns)
col_train_bis = list(train.columns)

col_train_bis.remove('SalePrice')

mat_train = np.matrix(train)
mat_test  = np.matrix(test)
mat_new = np.matrix(train.drop('SalePrice',axis = 1))
mat_y = np.array(train.SalePrice).reshape((1460,1))

prepro_y = MinMaxScaler()
prepro_y.fit(mat_y)

prepro = MinMaxScaler()
prepro.fit(mat_train)

prepro_test = MinMaxScaler()
prepro_test.fit(mat_new)

train = pd.DataFrame(prepro.transform(mat_train),columns = col_train)
test  = pd.DataFrame(prepro_test.transform(mat_test),columns = col_train_bis)

train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,0.235294,0.207668,0.033420,0.666667,0.500,0.949275,0.883333,0.12250,0.125089,0.0,...,0.000000,0.111517,0.000000,0.0,0.0,0.0,0.0,0.090909,0.50,0.241078
1,0.000000,0.255591,0.038795,0.555556,0.875,0.753623,0.433333,0.00000,0.173281,0.0,...,0.347725,0.000000,0.000000,0.0,0.0,0.0,0.0,0.363636,0.25,0.203583
2,0.235294,0.217252,0.046507,0.666667,0.500,0.934783,0.866667,0.10125,0.086109,0.0,...,0.000000,0.076782,0.000000,0.0,0.0,0.0,0.0,0.727273,0.50,0.261908
3,0.294118,0.191693,0.038561,0.666667,0.500,0.311594,0.333333,0.00000,0.038271,0.0,...,0.000000,0.063985,0.492754,0.0,0.0,0.0,0.0,0.090909,0.00,0.145952
4,0.235294,0.268371,0.060576,0.777778,0.500,0.927536,0.833333,0.21875,0.116052,0.0,...,0.224037,0.153565,0.000000,0.0,0.0,0.0,0.0,1.000000,0.50,0.298709


In [10]:
# List of features
COLUMNS = col_train
FEATURES = col_train_bis
LABEL = "SalePrice"

# Columns
feature_cols = FEATURES

# Training set and Prediction set with the features to predict
training_set = train[COLUMNS]
prediction_set = train.SalePrice

In [11]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from keras.layers.normalization import BatchNormalization
from sklearn.model_selection import KFold

def cross_val(parameters):
    kernel = parameters['kernel']
    activation = parameters['activation']
    rate = parameters['rate']
    decay = parameters['decay']
    epsilon = parameters['epsilon']
    l1, l2, l3, l4, l5 = parameters['layers']
    
    X = training_set[FEATURES]
    y = training_set[LABEL].values
    results = [0. for i in range(parameters['k'])]
    kf = KFold(n_splits=parameters['k'])
    iteration = 0
    for train_index, test_index in kf.split(X):
        print("Iteration: %d" % iteration)
        X_train = X.loc[train_index]
        X_test = X.loc[test_index]
        y_train = y[train_index]
        y_test = y[test_index]
        model = Sequential()
        model.add(Dense(l1, input_dim=36, kernel_initializer=kernel, activation=activation))
        model.add(Dense(l2, kernel_initializer=kernel, activation=activation))
        model.add(Dense(l3, kernel_initializer=kernel, activation=activation))
        model.add(Dense(l4, kernel_initializer=kernel, activation=activation))
        model.add(Dense(l5, kernel_initializer=kernel))
        model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adadelta(lr=rate, rho=decay, epsilon=epsilon))

        model.fit(np.array(X_train), np.array(y_train), epochs=parameters['epochs'], batch_size=parameters['bs'])
        results[iteration] = model.evaluate(np.array(X_test), np.array(y_test))
        iteration += 1
    return sum(results) / k

In [13]:
seed = 7
np.random.seed(seed)

rates = [0.1, 0.05, 0.01, 0.005, 0.001]
results = [None for i in range(len(rates))]

parameters = {}
parameters['kernel'] = 'normal'
parameters['activation'] = 'relu'
parameters['decay'] = 0.005
parameters['epochs'] = 200
parameters['bs'] = 20
parameters['layers'] = [100, 100, 25, 25, 1]
parameters['k'] = 5
parameters['epsilon'] = 1e-07

In [ ]:
for i in range(len(rates)):
    print("Test %d" % i)
    parameters['rate'] = rates[i]
    results[i] = cross_val(parameters)

In [ ]:
print(results)
best = results.index(min(results))
print(best)

In [23]:
kernel = parameters['kernel']
activation = parameters['activation']
rate = 0.001
decay = parameters['decay']
epsilon = parameters['epsilon']
l1, l2, l3, l4, l5 = parameters['layers']
model = Sequential()
model.add(Dense(50, input_dim=36, kernel_initializer='normal', activation='relu'))
model.add(Dense(50, kernel_initializer='normal', activation='relu'))
model.add(Dense(25, kernel_initializer='normal', activation='relu'))
model.add(Dense(25, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))
#model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adadelta(lr=rate, decay=decay, epsilon=epsilon))
model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adadelta())
#model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adam())
#model.compile(loss='mean_squared_error', optimizer=keras.optimizers.SGD())
#model.compile(loss='mean_squared_error', optimizer=keras.optimizers.RMSprop())
#model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Nadam())

In [24]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X = training_set[FEATURES]
y = training_set[LABEL].values
model.fit(np.array(X), np.array(y), epochs=100, batch_size=30)

Epoch 1/100
49/49 [==============================] - 0s 1ms/step - loss: 0.0504
Epoch 2/100
49/49 [==============================] - 0s 1ms/step - loss: 0.0520
Epoch 3/100
49/49 [==============================] - 0s 1ms/step - loss: 0.0516
Epoch 4/100
49/49 [==============================] - 0s 2ms/step - loss: 0.0524
Epoch 5/100
49/49 [==============================] - 0s 2ms/step - loss: 0.0531
Epoch 6/100
49/49 [==============================] - 0s 1ms/step - loss: 0.0512
Epoch 7/100
49/49 [==============================] - 0s 1ms/step - loss: 0.0535
Epoch 8/100
49/49 [==============================] - 0s 1ms/step - loss: 0.0517
Epoch 9/100
49/49 [==============================] - 0s 979us/step - loss: 0.0527
Epoch 10/100
49/49 [==============================] - 0s 969us/step - loss: 0.0535
Epoch 11/100
49/49 [==============================] - 0s 1ms/step - loss: 0.0532
Epoch 12/100
49/49 [==============================] - 0s 1ms/step - loss: 0.0521
Epoch 13/100
49/49 [=============

In [22]:
y_predict = model.predict(np.array(test))

def to_submit(pred_y,name_out):
    y_predict = list(itertools.islice(pred_y, test.shape[0]))
    y_predict = pd.DataFrame(prepro_y.inverse_transform(np.array(y_predict).reshape(len(y_predict),1)), columns = ['SalePrice'])
    y_predict = y_predict.join(ID)
    y_predict.to_csv(name_out + '.csv',index=False)
    
to_submit(y_predict, "team_jarvis2")